# Testing MCTS with Various Models
Here we will test MCTS first with the DKT model

In [37]:
#%load_ext autoreload
#%autoreload 2
#%reload_ext autoreload

from concept_dependency_graph import ConceptDependencyGraph
import data_generator as dg
import simple_mdp as sm

In [53]:
n_concepts = 5
concept_tree = sm.create_custom_dependency()
n_students = 100000
seqlen = 4
policy = 'random'
filename = 'test-n{}-l{}-{}.pickle'.format(n_students, seqlen, policy)

In [16]:
print ("Initializing synthetic data sets...")
dg.generate_data(concept_tree, n_students=n_students, seqlen=seqlen, policy=policy, filename="{}{}".format(dg.SYN_DATA_DIR, filename))
print ("Data generation completed. ")

Initializing synthetic data sets...
Generating data for 100000 students with behavior policy random and sequence length 4.
Data generation completed. 


In [54]:
import dynamics_model_class as dmc
import numpy as np
import dataset_utils

In [55]:
# test_model hidden=16
# test_model_small hidden=5
# test_model_mid hidden=10
dmodel = dmc.DynamicsModel(model_id="test_model", timesteps=seqlen, load_checkpoint=False)

Loading RNN dynamics model...
Directory path for tensorboard summaries: ../tensorboard_logs/test_model/
Checkpoint directory path: ../checkpoints/test_model/
Model loaded.


In [56]:
# load toy data
data = dataset_utils.load_data(filename='{}{}'.format(dg.SYN_DATA_DIR, filename))
input_data_, output_mask_, target_data_ = dataset_utils.preprocess_data_for_rnn(data)

In [57]:
train_data = (input_data_[:,:,:], output_mask_[:,:,:], target_data_[:,:,:])

In [59]:
dmodel.train(train_data, n_epoch=2, load_checkpoint=True)

Training Step: 9848  | total loss: 0.01581 | time: 22.564s
| Adam | epoch: 007 | loss: 0.01581 -- iter: 89984/90000
Training Step: 9849  | total loss: 0.01566 | time: 23.715s
| Adam | epoch: 007 | loss: 0.01566 | val_loss: 0.01598 -- iter: 90000/90000
--
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'


In [60]:
preds = dmodel.predict(input_data_[:1,:2, :])
print len(preds)
print len(preds[0])
print preds[0]

1
4
[[0.7309736013412476, 0.5147796273231506, 0.4931224286556244, 0.01285130437463522, 0.010480817407369614], [0.8807737827301025, 0.513401448726654, 0.49343299865722656, 0.006647326517850161, 0.0045877499505877495], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0]]


In [61]:
rnnsim = dmc.RnnStudentSim(dmodel)

In [62]:
from mcts import *
actix = 0
actvec = np.zeros((n_concepts,))
actvec[actix] = 1
act = StudentAction(actix, actvec)
print rnnsim.sequence
print rnnsim.sample_observations()

[]
None


In [63]:
rnnsim.advance_simulator(act, 1)


In [64]:
print rnnsim.sequence
print rnnsim.sample_observations()
print rnnsim.sample_reward()

[array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])]
[0.7309719920158386, 0.5084134936332703, 0.49522334337234497, 0.011797797866165638, 0.011552166193723679]
1.75795879308
